<a href="https://colab.research.google.com/github/nihalsuri/thermal-anomaly-detection/blob/main/deploy/rpi/rpi_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import timm
from PIL import Image 
from torchvision import transforms, models
import cv2

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_name = torch.cuda.get_device_name()
print(device_name)
print(device)

NVIDIA GeForce GTX 1660 Ti with Max-Q Design
cuda


In [3]:
resnet10t = timm.create_model('resnet10t', pretrained=True, num_classes=3)
resnet10t = resnet10t.cuda() if device else resnet10t
resnet10t

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
   

In [4]:
path_res = "C:/Users/Nihal/Desktop/resnet10t-timm-2.pt"
class_mapping = ['healthy', 'misalignment', 'rotor damage']

In [5]:
def preprocess(path): 
  img = cv2.imread(path)
  img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
  img = Image.fromarray(img)
  img = transforms.functional.crop(img, 190, 380 , 140, 230)
  to_tensor = transforms.ToTensor()
  img = to_tensor(img)
  return img 

In [6]:
model_resnet = resnet10t
model_resnet.load_state_dict(torch.load(path_res))

<All keys matched successfully>

In [7]:
input_test = preprocess("C:/Users/Nihal/Desktop/tests/rotor.png")
target_test = torch.tensor(int(2))
model_resnet.eval()
with torch.no_grad():
   input_test = torch.unsqueeze(input_test, 0) # adds another dimension 
   input_test = input_test.to(device) # send data to GPU
   predictions = model_resnet(input_test)
   predicted_index = predictions[0].argmax(0)

   predicted_test = class_mapping[predicted_index]
   expected_test = class_mapping[target_test]

print(predicted_test)
print(expected_test)

rotor damage
rotor damage
